# Programmatic Deployments

In [ ]:
!pip install kubernetes

In [ ]:
from os import path
import yaml
import time
from kubernetes import client, config
from kubernetes.client.rest import ApiException
from IPython.display import clear_output

In [ ]:
envs = [
    client.V1EnvVar("ENDPOINT", "minio.data:9000"),
    client.V1EnvVar("ACCESS_KEY_ID", "username"),
    client.V1EnvVar("ACCESS_KEY_SECRET", "password"),
    client.V1EnvVar("ENDPOINT_SSL", "false"),
]

In [ ]:
container = client.V1Container(
    name="compressor",
    image="apk8s/compressor:v0.0.1",
    env=envs,
    args=["-f=upload", 
          "-k=donors.csv", 
          "-t=processed"])

In [ ]:
podTmpl = client.V1PodTemplateSpec(
    metadata=client.V1ObjectMeta(
        labels={"app": "compress-donors"}
    ),
    spec=client.V1PodSpec(
        restart_policy="Never", 
        containers=[container]))

In [ ]:
job = client.V1Job(
    api_version="batch/v1",
    kind="Job",
    metadata=client.V1ObjectMeta(
        name="compress-donors"
    ),
    spec=client.V1JobSpec(
        template=podTmpl,
        backoff_limit=2)
    )

In [ ]:
config.load_incluster_config()
batch_v1 = client.BatchV1Api()

In [ ]:
resp = batch_v1.create_namespaced_job(
    body=job,
    namespace="data-lab")

In [ ]:
completed = False

while completed == False:
    time.sleep(1)
    
    try:
        resp = batch_v1.read_namespaced_job_status(
            name="compress-donors", 
            namespace="data-lab", pretty=False)
    except ApiException as e:
        print(e.reason)
        break

    clear_output(True)
    print(resp.status)
    
    if resp.status.conditions is None:
        continue
    
    if len(resp.status.conditions) > 0:
            
        clear_output(True)
        print(resp.status.conditions)
        
        if resp.status.conditions[0].type == "Failed":
            print("FAILED -- Pod Log --")
            core_v1 = client.CoreV1Api()
            pod_resp = core_v1.list_namespaced_pod(
                namespace="data-lab", 
                label_selector="app=compress-donors", 
                limit=1
            )
            
            log_resp = core_v1.read_namespaced_pod_log(
                name=pod_resp.items[0].metadata.name, 
                namespace='data-lab')
        
            print(log_resp)

        print("Removing Job...")
        resp = batch_v1.delete_namespaced_job(
            name="compress-donors",
            namespace="data-lab",
            body=client.V1DeleteOptions(
                propagation_policy='Foreground',
                grace_period_seconds=5))
        break